In [20]:
menuValidated = False # Uma variável para validar se um valor válido foi inserido no menu. Enquanto não for marcado como True, ficará em looping

saldo = 0
limite = 500
extrato = ""
numero_saques = 0
LIMITE_SAQUES = 3
depositos = []
saques = []

while True:
    while not menuValidated:
        menu = input('\nEntre com a opção desejada: \n [d] - Depositar \n [s] - Sacar \n [e] - Extrato \n [q] - sair\n')
        
        if menu not in ['d', 's', 'e', 'q']:
            print('\nPor gentileza, entre com uma das opções disponíveis.')
        else:
            menuValidated = True

    if menu == 'd':
        valor = input('\nInsira o valor do depósito: ')
        try:
            valor = float(valor)

            if valor <= 0:
                print('\nValor inválido.')
            else:
                saldo += valor
                depositos.append(valor)
                extrato += f"Depósito: R$ {valor:.2f}\n"
                print(f'\nDepósito realizado. Seu saldo é: R$ {saldo:.2f}')
                print(depositos)

        except ValueError:
            print("\nO depósito só aceita valores numéricos.")
            
    elif menu == 's':
        if numero_saques >= LIMITE_SAQUES:
            print('\nLimite de saques diário atingido.')
        else:
            valor = input('\nInsira o valor do saque: ')
            try:
                valor = float(valor)

                if valor <= 0:
                    print('\nValor inválido.')
                elif valor > limite:
                    print('\nO valor máximo de saque é 500.')
                elif valor > saldo:
                    print(f'\nVocê não tem fundos suficientes na sua conta: \nValor de Saque: R$ {valor:.2f} \nSaldo: R$ {saldo:.2f}')
                else:
                    saldo -= valor
                    saques.append(valor)
                    extrato += f"Saque: R$ {valor:.2f}\n"
                    numero_saques += 1
                    print(f'\nSaque realizado. Seu saldo é: R$ {saldo:.2f}')
                    print(saques)

            except ValueError:
                print("\nO saque só aceita valores numéricos.")

    elif menu == 'e':
        print("\n================ EXTRATO ================")
        print("Não foram realizadas movimentações." if not extrato else extrato)
        print(f"\nSaldo: R$ {saldo:.2f}")
        print("==========================================")

    elif menu == 'q':
        break

    menuValidated = False 



Entre com a opção desejada: 
 [d] - Depositar 
 [s] - Sacar 
 [e] - Extrato 
 [q] - sair
d

Insira o valor do depósito: 100

Depósito realizado. Seu saldo é: R$ 100.00
[100.0]

Entre com a opção desejada: 
 [d] - Depositar 
 [s] - Sacar 
 [e] - Extrato 
 [q] - sair
s

Insira o valor do saque: 50

Saque realizado. Seu saldo é: R$ 50.00
[50.0]

Entre com a opção desejada: 
 [d] - Depositar 
 [s] - Sacar 
 [e] - Extrato 
 [q] - sair
e

================ EXTRATO ================
Depósito: R$ 100.00
Saque: R$ 50.00


Saldo: R$ 50.00

Entre com a opção desejada: 
 [d] - Depositar 
 [s] - Sacar 
 [e] - Extrato 
 [q] - sair
d

Insira o valor do depósito: 977

Depósito realizado. Seu saldo é: R$ 1027.00
[100.0, 977.0]

Entre com a opção desejada: 
 [d] - Depositar 
 [s] - Sacar 
 [e] - Extrato 
 [q] - sair
e

================ EXTRATO ================
Depósito: R$ 100.00
Saque: R$ 50.00
Depósito: R$ 977.00


Saldo: R$ 1027.00


KeyboardInterrupt: Interrupted by user

In [14]:
saldo

39